In [ ]:
%matplotlib notebook

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pykep as pk
import pygmo as pg

import gtocprep as g
from math import pi, sin, cos, degrees, radians

### Orbital periods
What are the orbit periods at the starting position and at the moons?

T = 2pi * sqrt(a^3/mu)

In [ ]:
a_range = np.linspace(4*g.RADIUS_SATURN, 1000*g.RADIUS_SATURN, 1000)
T = 2 * pi * (a_range**3/g.MU_SATURN)**.5
T /= pk.DAY2SEC

fig1 = plt.figure(1)
ax = fig1.add_subplot(1, 1, 1)
ax.plot(a_range/g.RADIUS_SATURN, T)

planets = [g.enceladus, g.tethys, g.dione, g.rhea, g.titan]
for planet in planets:
    a = planet.orbital_elements[0]
    name = planet.name
    period = 2 * pi * (a**3/g.MU_SATURN)**.5 /pk.DAY2SEC
    plt.plot(a/g.RADIUS_SATURN, period, '.', color=g.pl2c[name])
    plt.text(a/g.RADIUS_SATURN, period, '  '+name, rotation=-30.)
plt.plot(a_range[-1]/g.RADIUS_SATURN, T[-1], '.', color='k')
plt.text(a_range[-1]/g.RADIUS_SATURN, T[-1], '  START', rotation=-30.)
ax.set_ylabel('$T$ [days]')
ax.set_xlabel('${R_{Saturn}}$')
pass

### Arriving to Titan from start state
Doing a Hohmann transfer? Or bi-elliptic? We can choose the 

In [ ]:
def hohmann(r1, r2, mu=1.0):
    a = 0.5 * (r1 + r2)
    vc1 = np.sqrt( mu / r1 )
    vp = np.sqrt( (2.0 * mu / r1) - ( mu / a) )
    va = np.sqrt( (2.0 * mu / r2) - ( mu / a) )
    vc2 = np.sqrt( mu / r2 )
    
    dv1 = np.abs(vp - vc1)
    dv2 = np.abs(vc2 - va)
    dv = dv1 + dv2
    return dv

hohmann(g.titan.orbital_elements[0], g.RADIUS_SATURN*1000, g.MU_SATURN)

In [ ]:
g.titan.orbital_elements[0]

### Choosing initial direction
How does the direction of velocity at the starting position influences initial orbital elements? I guess we need to minimize initial energy to help slowing down. Also going towards Saturn should help to get there fast.

-> Doesn't necessarily make sense, 3km/sec is lot higher than the escape velocity (1.12km/sec).

In [ ]:
(g.RADIUS_SATURN*1000/1000) / 3 / pk.DAY2SEC

In [ ]:
pk.planet.keplerian(g.REF_EPOCH, g.TITAN_ELEM, g.MU_SATURN, g.MU_TITAN, g.RADIUS_TITAN, g.RADIUS_TITAN*1.1, 'spacecraft')

In [ ]:
spacecraft = pk.planet.keplerian(g.REF_EPOCH,
                    [g.RADIUS_SATURN*629.376, 0, 0],
                    [1000, 1000, 0],
                    g.MU_SATURN, 1, 1, 1*1.1, 'spacecraft')
fig2 = plt.figure(2)
ax2 = fig2.gca(projection='3d')
pk.orbit_plots.plot_planet(spacecraft, ax=ax2, color='k', N=1000)

In [ ]:
(2*g.MU_SATURN/((g.RADIUS_SATURN)*1000))**.5/1000

In [ ]:
237948*1000/g.RADIUS_SATURN

### Playing with fly-bys

Considering arriving in the same plane, how Beta and the relative heading influences the output velocity magnitude?

In [ ]:
r_titan, v_titan = g.titan.eph(g.REF_EPOCH)
x_t = v_titan / np.linalg.norm(v_titan)
y_t = r_titan / np.linalg.norm(r_titan)


mu = g.MU_TITAN

phi = 0 * 2 * pi  # angle measured from titan velocity
magnitude = 3000  # m/s

def calculate_v_outs(values, rp, beta):
    v_outs = []
    for value in values:
    
        phi = radians(value)
    
        v_in = magnitude * (x_t*np.cos(phi) + y_t*np.sin(phi))
        v_out = pk.fb_prop(v_in, v_titan, rp, beta,mu)
        v_outs.append(np.linalg.norm(v_out))
    return v_outs

fig3 = plt.figure(3)
ax = fig3.add_subplot(1, 1, 1)    

rp = g.RADIUS_TITAN * 2
beta = 0.00*pi
values = np.linspace(-10, 370, 381)
v_outs1 = calculate_v_outs(values, rp, beta)
ax.plot(values, v_outs1, label='0')

beta = .25*pi
values = np.linspace(-10, 370, 381)
v_outs2 = calculate_v_outs(values, rp, beta)
ax.plot(values, v_outs2, label='45 deg')

beta = .5*pi
values = np.linspace(-10, 370, 381)
v_outs3 = calculate_v_outs(values, rp, beta)
ax.plot(values, v_outs3, label='90 deg')

ax.grid(True)
plt.legend(title='Beta angle')
ax.set_xlabel('realtive heading')
ax.set_ylabel('$|v_{out}|$')

So apparently
  * the biggest speed change is if we are staying in the plane (beta=90). 
  * If beta is small enough, the object only gains speed (get's a huge inclination as well), cannot break

In [ ]:
fig3 = plt.figure(4)
ax = fig3.add_subplot(1, 1, 1)    

beta = .5*pi

rp = g.RADIUS_TITAN * 2
values = np.linspace(-10, 370, 380)
v_outs1 = calculate_v_outs(values, rp, beta)
ax.plot(values, v_outs1, label='2')

rp = g.RADIUS_TITAN * 3
values = np.linspace(-10, 370, 380)
v_outs2 = calculate_v_outs(values, rp, beta)
ax.plot(values, v_outs2, label='3')

rp = g.RADIUS_TITAN * 4
values = np.linspace(-10, 370, 380)
v_outs3 = calculate_v_outs(values, rp, beta)
ax.plot(values, v_outs3, label='4')

ax.grid(True)
plt.legend(title='$R_{Saturn}$')
ax.set_xlabel('realtive heading')
ax.set_ylabel('$|v_{out}|$')

Nothing surprising here, the closer we are the bigger delta_v we get.